In [14]:
import numpy as np
import rasterio
import os
from scipy.spatial import KDTree
import time

In [15]:
DANGER_RADIUS = 5.0
TEXT_PROMPTS = ["tree", "house"]

In [16]:
try:
    masks = np.load('output_masks.npy')
    class_ids = np.load('output_class_ids.npy')

    DSM_PATH = "data/test_image3.tif"
    with rasterio.open(DSM_PATH) as src:
        dsm_data = src.read(1)
        (H, W) = dsm_data.shape

    print(f"{H}x{W}")
except FileNotFoundError:
    print("error")

520x779


In [17]:
labels_map = np.zeros((H, W), dtype=np.uint8)

for i, class_id in enumerate(class_ids):
    mask = masks[i]
    labels_map[mask] = class_id + 1

In [18]:
y_indices, x_indices = np.indices((H, W))

all_points = np.stack([
    x_indices.ravel(),
    y_indices.ravel(),
    dsm_data.ravel()
], axis=1)

all_labels = labels_map.ravel()

In [19]:
house_points = all_points[all_labels == 2] # 2 = House
tree_points = all_points[all_labels == 1]  # 1 = Tree

if len(house_points) == 0 or len(tree_points) == 0:
    print("Error. Points not found.")
else:
    print(f" Found house {len(house_points)} points and tree{len(tree_points)} points.")


 Found house 6107 points and tree116151 points.


In [20]:
if 'house_points' in locals() and 'tree_points' in locals():
    start_time = time.time()

    house_tree = KDTree(house_points)

    distance, _ = house_tree.query(tree_points, k=1)

    is_dangerous = (distance <= DANGER_RADIUS)

    num_danger_points = np.sum(is_dangerous)
    end_time = time.time()


In [21]:
if 'is_dangerous' in locals():

    tree_colors = np.full((len(tree_points), 3), [0, 255, 0], dtype=np.uint8)
    tree_colors[is_dangerous] = [255, 0, 0]

    house_colors = np.full((len(house_points), 3), [0, 0, 255], dtype=np.uint8)

    final_points = np.concatenate((house_points, tree_points))
    final_colors = np.concatenate((house_colors, tree_colors))

    num_final_points = len(final_points)
    max_plot_points = 1_000_000

    if num_final_points > max_plot_points:
        step = num_final_points // max_plot_points
        final_points = final_points[::step]
        final_colors = final_colors[::step]

    num_final_points = len(final_points)

    def save_to_ply_classified(filename, points, colors):
        num_points = len(points)
        print(f"Saving {num_points} point to {filename}...")

        data = np.hstack([points, colors])

        header = f"""ply
format ascii 1.0
element vertex {num_points}
property float x
property float y
property float z
property uchar red
property uchar green
property uchar blue
end_header
"""
        try:
            np.savetxt(
                filename,
                data,
                fmt="%f %f %f %d %d %d",
                header=header.strip(),
                comments=''
            )
        except Exception as e:
            print(f"error {e}")

    save_to_ply_classified('danger_point_cloud.ply', final_points, final_colors)

Saving 122258 point to danger_point_cloud.ply...
